# Lineare Regression am Beispiel von Immobilienpreisen in Boston

Regression wird genutzt, um Datensätze mit kontinuierlichen Werten als lineare Funktionen abzubilden. In diesem interaktiven Notebook wird anhand eines Beispiels erklärt, wie lineare Regression mithilfe der Machine Learning Bibliothek [scikit-learn](https://scikit-learn.org) implementiert werden kann.

In diesem Beispiel wird lineare Regression genutzt, um Anhand einer oder mehrerer Eigenschaften (Features) eines Datensatzes den mittleren Immobilienpreis in einer Region vorherzusagen. In der Regel werden mehrere solche Features auf einmal genutzt, um Vorhersagen zu treffen. Der Einfachheit halber wird hier pro Modell nur eines verwendet.

![Häuser in der Stadt Boston](https://proxy.duckduckgo.com/iu/?u=https%3A%2F%2Fthumbs.dreamstime.com%2Fx%2Fboston-houses-1004400.jpg&f=1)

## Programmbibliotheken importieren
In der folgenden ausführbaren Zelle werden die benötigten Python-Programmbibliotheken importiert. Nach dem Import stehen alle benötigten Funktionen zur Verfügung.

Codezellen können mit `Shift+Enter` ausgeführt werden.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## Datensatz laden
Als nächstes wird das [Boston Housing Prices Dataset](https://www.cs.toronto.edu/%7Edelve/data/boston/bostonDetail.html) von der University of Toronto geladen. Neben den Immobilienpreisen stehen zahlreiche weitere Attribute, wie zum Beispiel Verbrechensrate, Grad der Luftverschmutzung durch Stickoxide oder Schüler/Lehrer-Verhältnis. Später wird überprüft, ob diese Werte einen Anhaltspunkt zur Berechnung der Immobilienpreise geben können bzw. ob eine Korrelation vorliegt.

Die ersten fünf Zeilen des Datensatzes werden nach Ausführung der nächsten Codezelle ausgegeben. Eine Legende der einzelnen Spaltenabkürzungen findet sich auf der oben verlinkten Seite.

In [ ]:
# Das "Boston Houses Dataset" wird von scikit-learn geladen
boston_dataset = datasets.load_boston()

# Umwandeln des Datensarzes in einen übersichtlichen pandas-Dataframe
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)

# Erstellen einer Spalte für die Immobilienpreise
boston['MEDV'] = boston_dataset.target

# Anzeigen einer Vorschau des Datensatzes
boston.head()

## Daten untersuchen
Der Datensatz umfasst 506 unsortierte Einträge. Die Spalte `MEDV` enthält den mittleren Wert der Immobilienpreise (USD) in diesem Gebiet, geteilt durch 1000.

Mit dem Ausführen der folgenden Zelle wird die Verteilung der Immobilienpreise als Histogramm visualisiert. Die Preise sind annähernd normalverteilt um etwa 20000$, was erst einmal sehr wenig erscheint. Die Immobilienpreise stammen allerdings aus den 70er Jahren und die Inflationsentwicklung wurde nicht berücksichtigt.

In [ ]:
sns.distplot(boston['MEDV'], bins=30)
plt.show()

### Korrelation visualisieren
Verschiedene Spalten des Datensatzes scheinen Einfluss auf den mittleren Immobilienpreis (MEDV) in einer Region zu haben. Um diese Korrelationen festzustellen, hilft es, die Daten in zweidimensionalen "Scatter-Plots" anzeigen zu lassen. Die Streuung der Punkte signalisiert die  Stärke einer Korrelation.

In [ ]:
# Größe der Abbildung festlegen
plt.figure(figsize=(20, 5))

# Spalten zum Visualisieren auswählen
features = ['PTRATIO', 'RM', 'NOX']

# Auf der Y-Achse ist immer der mittlere Immobilienpreis
target = boston['MEDV']

# Plot erstellen
for i, col in enumerate(features):
    plt.subplot(1, len(features) , i+1)
    x = boston[col]
    y = target
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('MEDV')

#### _Aufgabe: andere Features untersuchen_
In der obigen Codezelle werden die Korrelationen der Spalten *LSTAT*, *RM* und *NOX* untersucht. Durch Austauschen dieser Werte in der Liste `features` können andere Spalten untersucht werden. Gibt es hier noch andere, gut sichtbare Korrelationen?

## Trainings- und Testdatensatz erstellen
Um ein Modell zu entwickeln, welches _generalisiert_, also auch für neue Daten einsetzbar ist, ist es wichtig, unterschiedliche Datensätze für Training und Evaluation des Modells zu benutzen. Mit der Funktion `train_test_split` bringt scikit-learn eine Funktion mit, die den Datensatz mit den Immobilienpreisen zufällig in Trainings und Testdatensatz aufteilt. Das hier gewählte Verhältnis ist 80/20.

In [ ]:
# Anteil des Testdatensatzes auswählen
test_size = 0.2

# Feature auswählen
feature = 'RM'

# Aufteilung erstellen
X_train, X_test, y_train, y_test = train_test_split(boston[feature], boston['MEDV'], test_size = test_size, random_state=5)

## Regressionsmodell mit `scikit-learn` erstellen
scikit-learn bringt eine ganze Reihe von [fertigen Regressionsmodellen](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) mit. Das Modell für die lineare Regression wird in der folgenden Codezelle mit einem einfachen Funktionsaufruf erzeugt.

In [ ]:
# Modell instanziieren
regr = linear_model.LinearRegression()

## Modell trainieren
Nun werden die Koeffizienten des Regressionsmodells für den Boston Housing Prices Datensatz angepasst. Dies geschieht in scikit-learn durch den Funktionsaufruf `regr.fit()` in der folgenden Codezelle.

Da nur ein Feature des Datensatzes verwendet wird, muss vorher noch die Dimension der Eingabedaten angepasst werden. 

In [ ]:
# Anpassen des Input-Formats
X_train = np.array(X_train).reshape(-1, 1)
X_test = np.array(X_test).reshape(-1, 1)

# Trainieren des Modells mit den Trainingsdaten
history = regr.fit(X_train, y_train)

In [ ]:
# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [ ]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

In [ ]:
# Plot outputs
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, color='crimson', linewidth=3)

plt.show()

## Aufgaben
* Wie ändert sich die mittlere quadratische Abweichung (Mean Squared Error), wenn andere Attribute (Features) im Trainingsdatensatz genutzt werden?

## Weiterführende Links
[Linear Regression Simulator](https://www.mladdict.com/linear-regression-simulator) ist eine interaktive Web-Applikation zur Simulation von linearer Regression. Datenpunkte können per Mausklick gesetzt werden und der Trainingsprozess per Gradientenverfahren kann schritt für Schritt nachvollzogen werden.